In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
import torch
from transformers import AutoModel, AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/media/souravsaini/Data/POP_OS/internship/notebooks_updated/notebooks/tokenizer_checklist.chk')

# Load the model
model = AutoModel.from_pretrained('/media/souravsaini/Data/POP_OS/internship/notebooks_updated/notebooks/llama-2-7b/checklist.chk')

# Function for zero-shot classification
def zero_shot_classification(text, labels):
    # Tokenize input text and labels
    inputs = tokenizer(text, return_tensors='pt')
    label_tokens = [tokenizer(label, return_tensors='pt') for label in labels]
    
    # Get model outputs for the input text
    outputs = model(**inputs)
    
    # Get embeddings for the input text and labels
    input_embeds = outputs.last_hidden_state[:, 0, :]  # Use CLS token
    label_embeds = [model(**label_token).last_hidden_state[:, 0, :] for label_token in label_tokens]
    
    # Calculate cosine similarity between input and label embeddings
    similarities = [torch.nn.functional.cosine_similarity(input_embeds, label_embed) for label_embed in label_embeds]
    
    # Get the label with the highest similarity
    predicted_label = labels[torch.argmax(torch.tensor(similarities))]
    
    return predicted_label

# Example usage
text = "The movie was fantastic!"
labels = ["positive", "negative"]
predicted_label = zero_shot_classification(text, labels)
print(f"Predicted label: {predicted_label}")

OSError: Incorrect path_or_model_id: '/media/souravsaini/Data/POP_OS/internship/notebooks_updated/notebooks/tokenizer_checklist.chk'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [2]:
model_name = 'facebook/llama-2-7b'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/home/ajeet/Sourav/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like facebook/llama-2-7b is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [3]:
texts = ["This is a positive text.", "This is a negative text."]
labels = [1, 0]

In [ ]:
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    features = outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
